<a href="https://colab.research.google.com/github/tuva-health/notebooks/blob/main/medicare_lds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook explores the patient population in the Medicare LDS SAF dataset.

# I. Import Data

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
from google.cloud import bigquery
client = bigquery.Client(project='medicare-lds-five-percent')
# client = bigquery.Client(project='tuva-claims-demo')

In [3]:
sql_query = ('''SELECT *
                FROM core.patient
                ''')
patient = client.query(sql_query).to_dataframe()

sql_query = ('''SELECT *
                FROM chronic_conditions.tuva_chronic_conditions_wide
                ''')
chronic_conditions = client.query(sql_query).to_dataframe()

In [5]:
patient.isna().sum()

patient_id           0
gender               0
race                 0
birth_date           0
death_date     3158164
death_flag           0
first_name     3296240
last_name      3296240
address        3296240
city           3296240
state              453
zip_code       3296240
phone          3296240
data_source          0
dtype: int64